In [1]:
##Data Query from https://www.junglescout.com/
## Product Price: between 20 and 100 USD
## Minimum Net Profit estimate: 15 USD
## Minimum monthly estimated sales: 200
## Maximum reviews: 50
## Maximum "Listing Quality Score" (LQS): 6 (Scale is 1-10)
## Exclude difficult categories: Electronics, Food
## Exclude "Top Brands"
## Include only items that JS marked as Fulfilled-by-Amazon (FBA)

In [2]:
import pandas as pd
import sqlite3
import altair as alt

In [3]:
from google.colab import files
uploaded = files.upload() ##import data

Saving products (1).db to products (1).db


In [5]:
## Data Cleaning

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)


# simple example of trimming whitespace from data elements
df = trim_all_columns(df)


In [6]:
def clean_rank_data(x):
    """ If the value is a string, then remove currency symbol, delimiters, and N.A.
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', '').replace('N.A.','0'))
    return(x)

In [7]:
# Clean data and convert to float data type
df['Rank'] = df['Rank'].apply(clean_rank_data).astype('float')

In [8]:
# Clean data and convert to float data type
df['Reviews'] = df['Reviews'].apply(clean_rank_data).astype('float')

In [9]:
# Clean data and convert to float data type
df['LQS'] = df['LQS'].apply(clean_rank_data).astype('float')

In [10]:
# Check Data Type in Rank
# See float data types in the column. 

df['Rank'].apply(type).value_counts()

<class 'float'>    5000
Name: Rank, dtype: int64

In [11]:
# Check Data Type in Est_Monthly_Revenue
# See different data types in the same column.  

df['Est_Monthly_Revenue'].apply(type)

3309    <class 'str'>
4891    <class 'str'>
3549    <class 'str'>
1584    <class 'str'>
2082    <class 'str'>
            ...      
344     <class 'str'>
2368    <class 'str'>
3047    <class 'str'>
739     <class 'str'>
2199    <class 'str'>
Name: Est_Monthly_Revenue, Length: 5000, dtype: object

In [12]:
# Check Data Type in Est_Monthly_Revenue
# See different data types in the same column. 

df['Est_Monthly_Revenue'].apply(type).value_counts()

<class 'str'>    5000
Name: Est_Monthly_Revenue, dtype: int64

In [13]:
def clean_Revenue_Sales_Fees_Net_data(x):
    """ If the value is a string, then remove currency symbol, delimiters, and N.A.
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', '').replace('N.A.','0').replace('--','0').replace('< ','-'))
    return(x)

In [14]:
# Clean data and convert to float data type
df['Est_Monthly_Revenue'] = df['Est_Monthly_Revenue'].apply(clean_Revenue_Sales_Fees_Net_data).astype('float')

In [15]:
# Clean data and convert to float data type
df['Est_Monthly_Sales'] = df['Est_Monthly_Sales'].apply(clean_Revenue_Sales_Fees_Net_data).astype('float')

In [16]:
# Clean data and convert to float data type
df['Fees'] = df['Fees'].apply(clean_Revenue_Sales_Fees_Net_data).astype('float')

In [17]:
# Clean data and convert to float data type
df['Net'] = df['Net'].apply(clean_Revenue_Sales_Fees_Net_data).astype('float')

In [18]:
# Clean data and convert to float data type
df['Rating'] = df['Rating'].apply(clean_Revenue_Sales_Fees_Net_data).astype('float')

In [19]:
# Clean data and convert to float data type
df['Price'] = df['Price'].apply(clean_Revenue_Sales_Fees_Net_data).astype('float')

In [ ]:
## below viz includes below features:
# 1) facet view following the 4 criteria defined by JS:
  ## 1) High demand and low competition  (Est_Monthly_Sales >200 and Reviews <50)
  ## 2) High demand and bad marketing (Est_Monthly_Sales > 200 and LQS <6)
  ## 3) Good return on Investment? (Est_Monthly_Sales >200 and price >20 and Net> 15)
  ## 4) Good demand + Poor quality products (Est_Monthly_Sales >200 and Rating < 3.7)
#2) multi panel scatter plot with linked brush
#3) configured tooltip with more product details
#4) red lines indicating preferred threshold (eg. sales>250)
#5) href encoding constructed from ASIN#, which allows users to click on any of the points to open the amazon page



In [49]:
## to drop off outliers, choose data that has 'Est_Monthly_Sales' <1500 & 'Reviews' <200)
source = df.loc[(df['Est_Monthly_Sales'] <1500) & (df['Reviews'] <200)]

## selector = alt.selection_single(empty='all', fields=['ASIN'])

brush = alt.selection(type='interval', resolve='global')

## chart1-High Demand and Low Competion
chart1 = alt.Chart(source,title="High Demand and Low Competion").transform_calculate(
    url='https://www.amazon.com/dp/' + alt.datum.ASIN
).mark_circle(size=60).encode(
    x='Reviews',
    y = alt.Y('Est_Monthly_Sales', scale=alt.Scale(domain=[0, 1600])),
    color=alt.condition(brush, 'ASIN:O', alt.ColorValue('whitesmoke'), legend=None),
    tooltip=['Product_Name','ASIN','Est_Monthly_Sales','Category','Reviews', 'LQS', 'Net','Price',]
).add_selection(
    brush).properties(
    height=300,
    width=300)

## chart2 - High Demand and Bad Marketing
chart2 = alt.Chart(source,title="High Demand and Bad Marketing").mark_circle(size=60).encode(
    x='LQS',
    y = alt.Y('Est_Monthly_Sales', scale=alt.Scale(domain=[0, 1600])),
    color=alt.condition(brush, 'ASIN:O', alt.ColorValue('whitesmoke'), legend=None),
    tooltip=['Product_Name','ASIN','Est_Monthly_Sales','Category','Reviews', 'LQS', 'Net','Price',]
).add_selection(
    brush).properties(
    height=300,
    width=300)

## chart3 - Good Return on Investment  
chart3 = alt.Chart(source,title="Good Return on Investment").mark_circle(size=60).encode(
    x='Price',
    y='Net',
    color=alt.condition(brush, 'ASIN:O', alt.ColorValue('whitesmoke'), legend=None),
    tooltip=['Product_Name','ASIN','Est_Monthly_Sales','Category','Reviews', 'LQS', 'Net','Price',]
).add_selection(
    brush).properties(
    height=300,
    width=300)

## chart4 - Good Demand & Poor Quality

chart4 = alt.Chart(source,title="Good Demand & Poor Quality").mark_circle(size=60).encode(
    x='Rating',
    y = alt.Y('Est_Monthly_Sales', scale=alt.Scale(domain=[0, 1600])),
    color=alt.condition(brush, 'ASIN:O', alt.ColorValue('whitesmoke'), legend=None),
    tooltip=['Product_Name','ASIN','Est_Monthly_Sales','Category','Reviews', 'LQS', 'Net','Price',]
).add_selection(
    brush).properties(
    height=300,
    width=300)

## line1 Est_Monthly_Sales >= 200
line1 = alt.Chart(pd.DataFrame({'y': [200]})).mark_rule(color='red').encode(y='y')
## line2 Reviews <=50
line2 = alt.Chart(pd.DataFrame({'x': [50]})).mark_rule(color='red').encode(x='x')
## line3 LQS <6
line3 = alt.Chart(pd.DataFrame({'x': [6]})).mark_rule(color='red').encode(x='x')
## line4 Price>20
line4 = alt.Chart(pd.DataFrame({'x': [20]})).mark_rule(color='red').encode(x='x')
## line5 Net>15
line5 = alt.Chart(pd.DataFrame({'y': [15]})).mark_rule(color='red').encode(y='y')
## line6 Rating<3.7
line6 = alt.Chart(pd.DataFrame({'x': [3.7]})).mark_rule(color='red').encode(x='x')


 
chart1 + line1 + line2|chart2 + line1 + line3|chart3 + line5 +line4|chart4 + line1 + line6

alt.HConcatChart(...)

# New Section